# `Практикум по программированию на языке Python`
<br>

## `Занятие 11: Основы Web-разработки`
<br><br>

### `Алексеев Илья (tg: voorhs)`

#### `ММП ВМК МГУ, 2024`

### `Мотивация. Игрушечный пример`

Представим ситуацию: вы решили сложную задачу (алгоритм поиска, обучение нейросети и т.д.). В нашем случае [задача с литкода](https://leetcode.com/problems/longest-palindromic-substring/description/).


Содержимое файла `solution.py`:
```python
def find_palindrome(s: str) -> str:
    input_str = s
    is_pal = [
        [False] * len(input_str) for _ in input_str
    ]  # is_pal[i][j] == True iff input_str[i:j+1] is palindromic

    for i in range(len(input_str)):
        is_pal[i][i] = True

    ans = input_str[0]
    for substring_len in range(2, len(input_str) + 1):
        cur_res = None
        for i in range(len(input_str) - substring_len + 1):
            j = i + substring_len - 1
            tmp_res = (input_str[i] == input_str[j]) and (
                substring_len <= 3 or is_pal[i + 1][j - 1]
            )
            is_pal[i][j] = tmp_res
            if tmp_res and cur_res is None:
                cur_res = input_str[i : j + 1]
        if cur_res is not None:
            ans = cur_res
    return ans

```

Что если вы хотите поделиться с миром своей работой? Нужно создать интерфейс взаимодействия! 

### `Python package`

Первое что необходимо сделать, это обернуть свое решение в Python API. Сделаем пакет, из которого можно импортировать функцию `find_palindrome`.


Организуем проект:
```bash
toy_example/
├── longest_palindrome
│   ├── __init__.py
│   └── solution.py
├── requirements.txt
├── server.py
└── cli.py
```

Здесь:
- `longest_palindrome/solution.py` --- код с решением задачи
- `longest_palindrome/__init__.py` --- init-файл, который объявляет директорию `longest_palindrome` пакетом
- `requirements.py` --- зависимости, используемые внутри `solution.py`

Содержимое файла `__init__.py`:
```python
from .solution import find_palindrome

__all__ = ["find_palindrome"]
```

Пример использования:
```python
from longest_palindrome import find_palindrome

find_palindrome("ababad")
```

### `Command-line interface (CLI) / Terminal user interface (TUI)`

Другой способ взаимодействия с вашим решением --- через терминал. Как насчет того, чтобы реализовать следующий интерфейс:

```bash
voorhs@maibenben:~$ longest_palindrome --input-word "ababad"
> babad
```

В точности такой интерфейс пока реализовать не сможем... Но вот, что можем сделать. Добавим в корень проекта файл `cli.py` со следующим содержимым:
```python
import argparse
from longest_palindrome import find_palindrome

def main():
    parser = argparse.ArgumentParser(description="Find the longest palindrome in a string.")
    parser.add_argument("--input-word", type=str, help="The input string")
    args = parser.parse_args()

    result = find_palindrome(args.input_word)
    print(f"Longest palindrome: {result}")

if __name__ == "__main__":
    main()
```

Структура проекта:
```bash
toy_example/
├── longest_palindrome
│   ├── __init__.py
│   └── solution.py
├── requirements.txt
├── server.py
├── cli.py
└── ui.py
```

Теперь можем взаимодействовать с нашим приложением через терминал:
```bash
python cli.py --input-word "ababad"
```

Такой способ взаимодействия доступен более широкому кругу пользователей, нежели взаимодействие через python api. Для него не нужно знать python и уметь импортировать. Достаточно просто написать команду в терминале, который стоит на любом компьютере.

### HTTP server

Ещё более общим способом взаимодействия с приложением являются HTTP-запросы. Они работают в формате "request--response". Вот простой пример, реализующий такой интерфейс:
```python
from fastapi import FastAPI
from longest_palindrome import find_palindrome

app = FastAPI()

@app.get("/longest-palindrome/{input_string}")
def longest_palindrome(input_string: str):
    return find_palindrome(input_string)
    

if __name__ == "__main__":
    import uvicorn
    uvicorn.run(app, host="127.0.0.1", port=8000)
```

Теперь можно провзаимодействовать с приложением, введя в браузере следующий адрес
```url
http://127.0.0.1:8000/longest-palindrome/ababad
```
В ответ получим веб-страницу, с единственным содержимым: `"ababa"`

О других способах создания HTTP-запросов поговорим далее в этом уроке (python requests, curl). Единственный вывод, который нужно сделать из данного игрушечного примера: HTTP --- это интефейс взаимодействия с приложением. 

Разговору о FastAPI будет посвящена вся лекция, которая состоится через неделю.


### `HTTP`

HTTP (HyperText Transfer Protocol) — это протокол, позволяющий получать различные ресурсы. Изначально, как следует из названия — для документов, но сейчас уже для передачи произвольных данных. Лежит в основе обмена данными в Web.

Это протокол клиент-серверного взаимодействия, что означает инициирование запросов к сереверу самим получателем. Итоговый документ может состоять из различных частей: текст, аудио/видео файлов, скриптов. Взаимодействие осущетвляется посредством обмена одиночными сообщенимями: запрос-ответ.

HTTP является протоколом прикладного уровня, который в качестве транспорта использует возможности другого протокола — TCP. Основное требование к транспортному протоколу — надёжность, то есть, гарантированная доставка сообщений. По этой причине не подходит другой распространённый протокол — UDP, который не гарантирует доставку сообщений.

#### Преимущества
- Прост и человекочитаем
- Расширяем
- Не имеет состояния (каждый запрос — в отрыве от остальных), следовательно, базово простой

#### Расширения
- Кэш — сервер может инструктировать клиента/прокси о том, что и как надолго можно кэшировать
- Ослабления ограничения источника — инструкции клиенту о том, что на загружаемой странице может содержаться информация с других доменов
- Аутентификация — для доступа к защищённой информации
- Прокси и туннелирование — сокрытие источника или получателя информации, кэширование для уменьшения нагрузки
- Сессии — расширение для сохранения состояния взаимодействия. Несколько механизмов, самый известный — cookies

### `Создание HTTP-запросов`

- browser
- curl
- python requests

Пример с обращением через браузер был скорее для наглядности. В реальной жизни такой способ взаимодействия используют для получения медиа-контента в виде веб-страниц, именемых "сайтами".

В более общем случае HTTP может передавать произвольную информацию. Чтобы её получать, можете воспользоваться двумя инструментами: командой `curl` в терминале или библиотекой `requests` в python.

Пример **CURL** (client URL).
```bash
curl 'http://127.0.0.1:8000/longest-palindrome/babad' \
```

### `Состав запроса`

- HTTP-метод: GET, POST, OPTIONS и т. д., определяющее операцию, которую клиент хочет выполнить
- Путь к ресурсу
- Версию HTTP-протокола
- Заголовки  (опционально)
- Тело (для некоторых методов, таких как POST)


```
GET / HTTP/1.1
Host: ya.ru
User-Agent: Python script
Accept: */*

```

### `Состав ответа`

- Версию HTTP-протокола
- HTTP код состояния, сообщающий об успешности запроса или причине неудачи
- Сообщение состояния -- краткое описание кода состояния
- HTTP заголовки
- Опционально: тело, содержащее пересылаемый ресурс


```
HTTP/1.1 200 Ok
Cache-Control: no-cache,no-store,max-age=0,must-revalidate
Content-Length: 59978
Content-Type: text/html; charset=UTF-8
Date: Thu, 29 Apr 2021 03:48:39 GMT
Set-Cookie: yp=1622260119.ygu.1; Expires=Sun, 27-Apr-2031 03:48:39 GMT; Domain=.ya.ru; Path=/
```

### `Типы запросов`

Типы запросов в какой-то степени просто договорённость о семантике. Никто не мешает пользоваться только одним типом. Но глобально типы призваны определить что именно требуется при обращении к одному и тому же ресурсу. Каждый конкретный ресурс может поддерживать только часть методов.

- GET — запрашивает представление ресурса. Запросы с использованием этого метода могут только извлекать данные
- HEAD — запрашивает ресурс так же, как и метод GET, но без тела ответа
- POST — используется для отправки сущностей к определённому ресурсу. Часто вызывает изменение состояния или какие-то побочные эффекты на сервере
- PUT — заменяет все текущие представления ресурса данными запроса
- DELETE — удаляет указанный ресурс
- CONNECT — устанавливает "туннель" к серверу, определённому по ресурсу
- OPTIONS — используется для описания параметров соединения с ресурсом
- TRACE — выполняет вызов возвращаемого тестового сообщения с ресурса (например, для отладки)
- PATCH — используется для частичного изменения ресурса

### `Заголовки`

- Authentication (WWW/Proxy-Authenticate, Authorization, Proxy-Authorization)
- Caching (Cache-control, Age, Expires)
- Client hints
- Conditionals
- Connection management (Connection, Keep-Alive)
- Cookies
- Message body information (Content-Length, Content-Type, Content-Encoding)
- Request context (Host, Referer, User-Agent)
- Response context
- Security (Referrer-Policy)
- WebSockets


### `Коды`

- Информационные (100 - 199)
- Успешные (200 - 299)
- Перенаправления (300 - 399)
- Клиентские ошибки (400 - 499)
- Серверные ошибки (500 - 599)

```
200 OK
302 Found
400 Bad Request
401 Unauthorized
404 Not Found
500 Internal Server Error
503 Service Unavailable
```

### `HTTPS`

- HTTPS не является отдельным протоколом передачи данных, а представляет собой расширение протокола HTTP с надстройкой шифрования
- передаваемые по протоколу HTTP данные не защищены, HTTPS обеспечивает конфиденциальность информации путем ее шифрования
- HTTP использует порт 80, HTTPS — порт 443

#### Сетевое обеспечение защищённого протокола

SSL — Secure Sockets Layer: "надстройка" на несколькими слоями сетевой модели

TLS — Transport Layer Security: развитие SSL

Обеспечивают шифрование и поддержку сертификатов

Принцип работы:
- С помощью ассиметричного шифрования устанавливается ключ соединения и передаётся сессионный ключ
- Всё дальнейшее общение шифруется уже сессионным ключом

#### Асимметричное шифрование

Называется так, потому что передающая сторона может только зашифровать (но не расшифровать) данные.

Асимметричный ключ — ключ, имеющий две составляющие: публичную и частную (закрытую). Публичный ключ доступен любому. Частный (закрытый) известен только владельцу. Если браузер хочет отправить сообщение, то он находит публичный ключ сервера, шифрует сообщение и отправляет на сервер. Далее сервер расшифровывает полученное сообщение с помощью своего частного ключа. Чтобы ответить пользователю, сервер делает те же самые действия: поиск публичного ключа собеседника, шифрование, отправка

Следовательно, для двустороннего общения требуется 2 пары ключей.

Алгоритмы ассиметричного шифрования более ресурсоёмкие, поэтому обычно только первичная установка соединения производится с его помощью, далее стороны договариваются о симметричном сессионном ключе и дальшейшее общение ведётся с помощью симметричного шифрованиия

#### Симметричное шифрование

В этом случае у обеих сторон есть один ключ, с помощью которого они и передают данные. Основная проблема — как договориться об этом ключе, чтобы знали только две стороны. И один из способов (не единственный) — с помощью ассиметричных алгоритмов

### `Python Web-clients`

Стандартная библиотека `urllib`

Реализует все необходимые методы и примитивы, но требует много вспомогательной работы

In [ ]:
import json
import urllib.request
ur = urllib.request.urlopen('https://postman-echo.com/get?foo=bar&foo1=bar1')
print(ur.code)
content = json.loads(ur.read())
print(json.dumps(content, indent=4, sort_keys=True))

In [ ]:
import json
from urllib import request, parse
data = parse.urlencode({ 'foo': 'bar' })
data = data.encode()
req = request.Request('https://postman-echo.com/post', method="POST", data=data)
ur = request.urlopen(req)
print(ur.code)
# print(ur.read())
content = json.loads(ur.read())
print(json.dumps(content, indent=4, sort_keys=True))

Библиотека `requests`

Инкапсулирует значительную часть вспомогательной обработки, позволяя сосредоточиться непосредственно на логике

Установка:
`pipenv install requests`

Запрос GET

In [ ]:
import requests

r = requests.get('https://postman-echo.com/get', params={'foo': 'bar'}, headers={'user-agent': 'Python Script'})
print(r.status_code)
content = json.loads(r.content)
print(json.dumps(content, indent=4, sort_keys=True))


Запрос POST

In [ ]:
import requests

r = requests.post('https://postman-echo.com/post', json={'foo': 'bar'},headers = {'user-agent': 'Python Script', 'referer': 'google.com'})
print(r.status_code)
content = json.loads(r.content)
print(json.dumps(content, indent=4, sort_keys=True))